In [34]:
from math import sin, cos,asin, degrees,sqrt, atan2, radians
import json
import pandas as pd
import numpy as np
import csv
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [48]:
cabeceras = pd.read_csv('estaciones_dentro_bicimad.csv', sep=";", header=0)
bicimad_df = pd.read_csv('bicimad_stations.csv', sep=";", header=0) 

## 4.3.1 Clasificación de las líneas que empiecen y terminen dentro de la zona

In [37]:
def get_lines(df):
    iguales = df.groupby('id_line')
    res = iguales.filter(lambda x: len(x) >= 2)
    return res

In [38]:
cabeceras_biciMAD = get_lines(cabeceras).sort_values(['id_line','type_header'])

##### Número de lineas encontradas

In [39]:
cabeceras_biciMAD.groupby('id_line').size().count()

21

In [43]:
cabeceras_biciMAD

,id_line,label,type_header,id_header,name_header,latitude,longitude
126,1,1,headerA,4514,Cristo Rey,40.438475,-3.717931
174,1,1,headerB,273,Prosperidad,40.443801,-3.674896
59,3,3,headerA,1885,Puerta de Toledo,40.406352,-3.711462
146,3,3,headerB,1855,San Amaro,40.455057,-3.697000
127,12,12,headerA,4541,Cristo Rey,40.439460,-3.715913
167,12,12,headerB,779,Marqués de Zafra,40.425278,-3.661794
143,19,19,headerA,5167,Plaza de Cataluña,40.448658,-3.678621
150,19,19,headerB,1171,Legazpi,40.392147,-3.694757
56,26,26,headerA,1919,Tirso de Molina,40.412547,-3.704619
142,26,26,headerB,2223,Diego de León,40.434293,-3.674781


## 4.3.2 Selección de estaciones de bicimad que se encuentran alrededor de las cabeceras de autobuses

In [8]:
def calcular_distancia(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2( sqrt(a), sqrt(1-a) )
    return R * c * 1000

In [9]:
def loop_distancia(latitud,longitud,radio,df):
    res = []
    for index, row in df.iterrows():
        distancia = calcular_distancia(latitud,longitud,row.latitude,row.longitude) 
        if distancia <= radio:
            res.append(row)
    return pd.DataFrame(res) 

In [10]:
def calcular_distancia_con_estacion_bicimad(latitude, longitude,radio):
    return loop_distancia(latitude, longitude,radio,bicimad_df)

In [11]:
def estaciones_alrededor_cabeceras(radio,df):
    list_aux = []
    for index, row in df.iterrows():
        res1 = calcular_distancia_con_estacion_bicimad(row.latitude,row.longitude,radio)
        if res1.shape[0] > 0:
            for index, r in res1.iterrows():
                row_aux = row
                row_aux['a'] = r.id
                list_aux.append(row_aux.tolist())
        else:
            row_aux = row
            row_aux['a'] = 'NA'
            list_aux.append(row_aux.tolist())
    return pd.DataFrame(list_aux,columns=["id_line","label","type_header","id_header","name_header","latitude","longitude","id_bicimad"])

##### Análisis con diferentes distancias de radio

##### 100 metros

In [12]:
DISTANCIA = 100
bicimas_alrededor_total = estaciones_alrededor_cabeceras(DISTANCIA,cabeceras_biciMAD)
result = get_lines(bicimas_alrededor_total[bicimas_alrededor_total.id_bicimad != 'NA'])

##### Número de cabeceras encontradas

In [13]:
result.count()[0]

17

#### Número de lineas encontradas

In [88]:
result.groupby('id_line').size().count()

6

##### 200 metros

In [89]:
DISTANCIA = 200
bicimas_alrededor_total = estaciones_alrededor_cabeceras(DISTANCIA,cabeceras_biciMAD)
result = get_lines(bicimas_alrededor_total[bicimas_alrededor_total.id_bicimad != 'NA'])

#### Número de cabeceras encontradas

In [90]:
result.count()[0]

56

#### Número de lineas encontradas

In [91]:
result.groupby('id_line').size().count()

19

##### 250 metros

In [92]:
DISTANCIA = 250
bicimas_alrededor_total = estaciones_alrededor_cabeceras(DISTANCIA,cabeceras_biciMAD)
result = get_lines(bicimas_alrededor_total[bicimas_alrededor_total.id_bicimad != 'NA'])

#### Número de cabeceras encontradas

In [93]:
result.count()[0]

75

#### Número de lineas encontradas

In [94]:
result.groupby('id_line').size().count()

21

##### 300 metros

In [14]:
DISTANCIA = 300
bicimas_alrededor_total = estaciones_alrededor_cabeceras(DISTANCIA,cabeceras_biciMAD)
result = get_lines(bicimas_alrededor_total[bicimas_alrededor_total.id_bicimad != 'NA'])

C:\Users\Arturo\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


#### Número de cabeceras encontradas

In [15]:
bicimas_alrededor_total.count()[0]

94

#### Número de lineas encontradas

In [16]:
bicimas_alrededor_total.groupby('id_line').size().count()

21

### Rutas de bicicleta encontradas con al menos una estacion de bici cerca de una cabecera de línea

In [24]:
estaciones_bicimad_totales = get_lines(bicimas_alrededor_total[bicimas_alrededor_total.id_bicimad != 'NA'])
estaciones_bicimad_totales_a = estaciones_bicimad_totales[estaciones_bicimad_totales.header == 'headerA']
estaciones_bicimad_totales_b = estaciones_bicimad_totales[estaciones_bicimad_totales.header == 'headerB']

C:\Users\Arturo\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [25]:
estaciones_bicimad_totales = estaciones_bicimad_totales_a.merge(estaciones_bicimad_totales_b,how='inner',on=['id_line'])
estaciones_bicimad_totales.columns = estaciones_bicimad_totales.columns.str.replace("_x", "_origen")
estaciones_bicimad_totales.columns = estaciones_bicimad_totales.columns.str.replace("_y", "_destino")

In [26]:
estaciones_bicimad_totales = estaciones_bicimad_totales.sort_values(['id_line','id_bicimad_origen','id_bicimad_destino'])
estaciones_bicimad_totales.drop(['label_origen','header_origen','label_destino','header_destino'], axis=1, inplace=True)

In [27]:
estaciones_bicimad_totales.head()

,id_line,id_header_origen,name_header_origen,latitude_origen,longitude_origen,id_bicimad_origen,id_header_destino,name_header_destino,latitude_destino,longitude_destino,id_bicimad_destino
0,1,4514,Cristo Rey,40.438475,-3.717931,160,273,Prosperidad,40.443801,-3.674896,204
1,3,1885,Puerta de Toledo,40.406352,-3.711462,49,1855,San Amaro,40.455057,-3.697000,141
2,12,4541,Cristo Rey,40.439460,-3.715913,160,779,Marqués de Zafra,40.425278,-3.661794,193
3,12,4541,Cristo Rey,40.439460,-3.715913,211,779,Marqués de Zafra,40.425278,-3.661794,193
4,19,5167,Plaza de Cataluña,40.448658,-3.678621,148,1171,Legazpi,40.392147,-3.694757,135


In [28]:
estaciones_bicimad_totales.to_csv('estaciones_bicimad_totales.csv', sep=';', encoding='utf-8',index=False)

##### Número de rutas encontradas con un radio de 300 metros

In [29]:
estaciones_bicimad_totales.count()[0]

106